In [ ]:
from flightradar24.api \
import FlightRadar24API
import folium

# 航空機情報を取得するエリア
bounds="60,22,113,176" 
fr_api=FlightRadar24API()
flights=fr_api.get_flights(
             bounds=bounds)
fmap = folium.Map(
  location=[35,135],zoom_start=3)
for f in flights:
    folium.Marker(# マーカープロット
 location=[f.latitude,f.longitude],
 icon=folium.Icon(icon="plane") 
    ).add_to(fmap)
fmap            # 地図表示

In [ ]:
import time
from datetime import datetime

flights = []; timeStamps=[]
for i in range(60*6*4): # 10秒に１回、６０分の取得を続ける
    flights.append(fr_api.get_flights(bounds=bounds))
    timeStamps.append(datetime.now())
    time.sleep(10)

In [ ]:
import numpy as np
# (lat,lon,heading,l)の単位は(deg,deg,deg,m)
def dstLatLon(lat, lon, heading, l): 
    lat0=l/(40000*1000)*360 # 地球１周(m)/360°
    lon0=l/(40000*1000)*360/np.cos(np.deg2rad(lat))
    lat0=lat0*np.cos(np.deg2rad(heading))
    lon0=lon0*np.sin(np.deg2rad(heading))
    return lat+lat0, lon+lon0

In [ ]:
f = open('flightsHaneda2.kml', 'w')
f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
f.write("<kml xmlns='http://earth.google.com/kml/2.2'>\n")
f.write("<Document>\n   <name>flight</name>\n")
for i, t in enumerate(timeStamps):
    fs = flights[i]
    for fl in fs:
        lat_d, lon_d = dstLatLon(fl.latitude, fl.longitude,
                                 fl.heading, fl.ground_speed*5)
        alt = fl.altitude*0.3048 # 飛行高度（フィート）をメートルに
        f.write("<Placemark>\n        <TimeSpan>\n          <begin>") 
        +'%04i-%02i-%02iT%02i:%02i:%02iZ'%(
            t.year, t.month, t.day, t.hour, t.minute, t.second)
        +"</begin>\n        </TimeSpan>\n")        
        f.write("   <Style>\n   <LineStyle>\n")
        f.write("   <color>80ffff00</color>\n") # 色順はABGR
        f.write("   <width>5</width>\n   </LineStyle>\n")
        f.write("   </Style>\n   <LineString>\n")
        f.write("       <extrude>0</extrude>\n")
        f.write("       <altitudeMode>absolute</altitudeMode>\n")
        f.write("        <coordinates>" 
        +str(fl.longitude)+","+str(fl.latitude)+","+str(alt)
        +" "+str(lon_d)+","+str(lat_d)+","+str(alt)+"</coordinates>\n")    
        f.write("   </LineString>\n</Placemark>\n")    
f.write("</Document></kml>\n"); f.close()